In [1]:
import numpy as np
import pandas as pd
import nltk
import re

In [2]:
data=pd.read_csv('emails.csv')
data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [3]:
data['spam'][1369]

0

In [4]:
text=data['text'].to_list()
len(text)

5728

In [5]:
spam=len(data[data['spam']==1])
non_spam=len(data[data['spam']==0])
print('Number of spam emails are : ',spam)
print('Numer of non spam emails are ', non_spam)
print('Do spam+non_spam comprise the entire dataset :', (spam+non_spam)==len(text))

Number of spam emails are :  1368
Numer of non spam emails are  4360
Do spam+non_spam comprise the entire dataset : True


# 0. CREATING THE DATASETS

In [6]:
train_pos=text[:1000]
test_pos=text[1000:1369]
train_neg=text[1369:4000]
test_neg=text[4000:]
train_x=train_pos+train_neg
test_x=test_pos+test_neg

In [28]:
train_y=np.append(np.ones((len(train_pos),1)),np.zeros((len(train_neg),1)),axis=0)
test_y=np.append(np.ones((len(test_pos),1)),np.zeros((len(test_neg),1)),axis=0)
print('Len(train_x)=Len(train_y) : ',len(train_x)==len(train_y))
print('Len(test_x)=Len(test_y) : ',len(test_x)==len(test_y))
train_y.shape

Len(train_x)=Len(train_y) :  True
Len(test_x)=Len(test_y) :  True


(3631, 1)

In [8]:
from nltk.corpus import stopwords
urlregex=r'^https?:\/\/.*'
def preprocess(text):
    #text->a string 
    stemmer=nltk.PorterStemmer()
    text=text.lower()
    re.sub(urlregex,'URL',text)
    word_l=text.split()
    word_l=[w for w in word_l if w not in stopwords.words('english')]
    return word_l  

In [9]:
text='My name is khan and i am not a terrorist'
ans=preprocess(text)
print(ans)

['name', 'khan', 'terrorist']


   # 1. LOGISTIC REGRESSION
   

In [74]:
def sigmoid(z):
    return 1/(1+np.exp(-1*z))

In [75]:
ans=sigmoid(0)
print(ans)

0.5


In [12]:
def build_freq(train_x,train_y):
    #train_x->list of strings 
    #train_y->corresponding label
    #output : freq:-> a dictionary which maps (word,label):count
    freq={}
    train_y=np.squeeze(train_y).tolist()
    for tweet,y in zip(train_x,train_y):
        word_l=preprocess(tweet)
        for word in word_l:
            pair=(word,y)
            if pair in freq.keys():
                freq[pair]+=1
            else:
                freq[pair]=1
    return freq

In [16]:
freqs=build_freq(train_x,train_y)

In [17]:
print(freqs)

{('subject:', 1.0): 1000, ('naturally', 1.0): 6, ('irresistible', 1.0): 2, ('corporate', 1.0): 40, ('identity', 1.0): 65, ('lt', 1.0): 20, ('really', 1.0): 65, ('hard', 1.0): 41, ('recollect', 1.0): 14, ('company', 1.0): 604, (':', 1.0): 3284, ('market', 1.0): 221, ('full', 1.0): 115, ('suqgestions', 1.0): 8, ('information', 1.0): 550, ('isoverwhelminq', 1.0): 3, (';', 1.0): 373, ('good', 1.0): 150, ('catchy', 1.0): 14, ('logo', 1.0): 137, (',', 1.0): 7986, ('stylish', 1.0): 7, ('statlonery', 1.0): 8, ('outstanding', 1.0): 43, ('website', 1.0): 336, ('make', 1.0): 355, ('task', 1.0): 14, ('much', 1.0): 131, ('easier', 1.0): 27, ('.', 1.0): 14124, ('promise', 1.0): 35, ('havinq', 1.0): 4, ('ordered', 1.0): 28, ('iogo', 1.0): 16, ('automaticaily', 1.0): 2, ('become', 1.0): 92, ('world', 1.0): 128, ('ieader', 1.0): 8, ('isguite', 1.0): 7, ('ciear', 1.0): 4, ('without', 1.0): 144, ('products', 1.0): 134, ('effective', 1.0): 68, ('business', 1.0): 602, ('organization', 1.0): 29, ('practicab

In [188]:
def extract_features(freqs,tweet):
    #tweet->a single tweet
    word_l=preprocess(tweet)
    x=np.zeros((1,3))
    x[0,0]=1
    for word in word_l:
        x[0,1]+=freqs.get((word,1),0)
        x[0,2]+=freqs.get((word,0),0)
        
    return x

In [189]:
text='my name is khan and i am not a terrorist '
extract_features(freqs,text)

array([[  1., 181., 327.]])

In [190]:
#training your model
X=np.zeros((len(train_y),3))
for i in range(len(train_y)):
    X[i,:]=extract_features(freqs,train_x[i])


In [191]:
Y=train_y

In [192]:
X_test=np.zeros((len(test_y),3))
for i in range(len(test_y)):
    X[i,:]=extract_features(freqs,test_x[i])

In [193]:
Y_test=test_y
print(Y_test.shape)
print(Y.shape)


(2097, 1)
(3631, 1)


In [194]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(
lr.fit(X,Y.ravel())
pred=lr.predict(X_test)
pred=pred.reshape(-1,1)
print(np.mean(pred==Y_test))

SyntaxError: invalid syntax (<ipython-input-194-5d625fbbea94>, line 4)

In [ ]:
print(pred.shape)